In [28]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, average_precision_score
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score, accuracy_score, f1_score
from sklearn.metrics import roc_curve, precision_recall_curve, average_precision_score
import matplotlib.pyplot as plt
import seaborn as sns

In [29]:
def plot_and_save_confusion_matrix(y_true, y_pred, title, filename):
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(6, 5))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False)
    plt.title(title)
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')
    plt.tight_layout()
    plt.savefig(filename)
    plt.close()


In [30]:


def extract_clinical_features(patient_data):
    """
    Extract clinically relevant features from patient time-series data,
    choosing the most appropriate function for each variable.
    """
    features = {}
    
    # Static variables (use as is)
    static_vars = ['Age', 'Gender', 'Height', 'Weight']
    for var in static_vars:
        non_null = patient_data[var].dropna()
        features[var] = non_null.iloc[0] if not non_null.empty else np.nan
    
    # Define which function to use for each clinical variable
    variable_functions = {
        # Vital Signs
        'HR': 'max',                # Maximum heart rate (tachycardia)
        'SysABP': 'min',            # Minimum systolic blood pressure (hypotension)
        'DiasABP': 'min',           # Minimum diastolic blood pressure
        'MAP': 'mean',              # Mean arterial pressure (average)
        'RespRate': 'max',          # Maximum respiratory rate (distress)
        'Temp': 'max',              # Maximum temperature (fever)
        'SaO2': 'min',              # Minimum oxygen saturation (hypoxia)
        'NISysABP': 'min',          # Non-invasive systolic (minimum)
        'NIDiasABP': 'min',         # Non-invasive diastolic (minimum)
        'NIMAP': 'mean',            # Non-invasive MAP (average)
        
        # Lab Values
        'BUN': 'max',               # Maximum blood urea nitrogen (kidney)
        'Creatinine': 'max',        # Maximum creatinine (kidney)
        'Glucose': 'mean',          # Average glucose
        'HCO3': 'last',             # Last bicarbonate
        'HCT': 'min',               # Minimum hematocrit (anemia)
        'K': 'last',                # Last potassium
        'Mg': 'last',               # Last magnesium
        'Na': 'last',               # Last sodium
        'Platelets': 'min',         # Minimum platelets (bleeding risk)
        'WBC': 'max',               # Maximum white blood cells (infection)
        'Lactate': 'max',           # Maximum lactate (tissue perfusion)
        'pH': 'min',                # Minimum pH (acidosis)
        'PaCO2': 'last',            # Last CO2
        'PaO2': 'min',              # Minimum O2 (hypoxemia)
        'Albumin': 'min',           # Minimum albumin
        'ALT': 'max',               # Maximum ALT (liver)
        'AST': 'max',               # Maximum AST (liver)
        'ALP': 'max',               # Maximum alkaline phosphatase (liver)
        'Bilirubin': 'max',         # Maximum bilirubin (liver)
        'Cholesterol': 'mean',      # Average cholesterol
        'TroponinI': 'max',         # Maximum troponin I (cardiac damage)
        'TroponinT': 'max',         # Maximum troponin T (cardiac damage)
        
        # Treatment Variables
        'MechVent': 'max',          # Whether ventilation was ever used (0/1)
        'Urine': 'sum',             # Total urine output
        'FiO2': 'max',              # Maximum oxygen support needed
        'GCS': 'min'                # Minimum Glasgow Coma Scale (worst neuro status)
    }
    
    # Process each variable with its appropriate function
    for var, func in variable_functions.items():
        non_null = patient_data[var].dropna()
        
        if non_null.empty:
            features[var] = np.nan
            continue
            
        if func == 'mean':
            features[var] = non_null.mean()
        elif func == 'max':
            features[var] = non_null.max()
        elif func == 'min':
            features[var] = non_null.min()
        elif func == 'last':
            last_idx = patient_data[var].last_valid_index()
            features[var] = patient_data.loc[last_idx, var] if last_idx is not None else np.nan
        elif func == 'sum':
            features[var] = non_null.sum()
    
    return features

In [31]:

def extract_advanced_clinical_features(patient_data):
    """
    Extract advanced clinically relevant features from patient time-series data,
    including original features plus up to 15 additional time-series features.
    All features are guaranteed to have numeric values (no NaNs).
    
    Args:
        patient_data: DataFrame containing patient time-series data
        
    Returns:
        Dictionary of extracted features with no NaN values
    """
    import numpy as np
    import pandas as pd
    
    # First get all the original features
    features = extract_clinical_features(patient_data)
    
    # Make sure we have a time column - if not, create one based on index
    if 'Hours' in patient_data.columns and 'Time' not in patient_data.columns:
        ts_data = patient_data.copy()
        ts_data = ts_data.rename(columns={'Hours': 'Time'})
        print("su")
    else:
        ts_data = patient_data
    
    # Define the critical vital signs for time analysis
    critical_vitals = ['HR', 'SysABP', 'RespRate', 'GCS', 'SaO2']
    
    # 1. Vital sign variability (standard deviation over time) - 5 features
    for var in critical_vitals:
        feature_name = f"{var}_variability"
        non_null = ts_data[var].dropna()
        if len(non_null) >= 3:
            features[feature_name] = non_null.std()
        else:
            # Default: 0 variability if not enough data points
            features[feature_name] = 0.0
    
    # 2. Trend features (slope of key vitals over time) - 5 features
    for var in critical_vitals:
        feature_name = f"{var}_trend"
        non_null_df = ts_data[['Time', var]].dropna()
        if len(non_null_df) >= 3:
            # Simple linear regression to get slope
            try:
                x = non_null_df['Time'].values
                y = non_null_df[var].values
                slope = np.polyfit(x, y, 1)[0]
                features[feature_name] = slope
            except:
                # Default: 0 slope (no change) if calculation fails
                features[feature_name] = 0.0
        else:
            # Default: 0 slope (no change) if not enough data points
            features[feature_name] = 0.0
    
    # 3. Shock index (HR/SBP ratio) - clinical marker of circulatory shock - 2 features
    # Initialize with default values
    features['ShockIndex_max'] = 0.0
    features['ShockIndex_trend'] = 0.0
    
    if 'HR' in ts_data.columns and 'SysABP' in ts_data.columns:
        # Get rows with valid data for both HR and SysABP
        shock_index_df = ts_data[['HR', 'SysABP']].dropna()
        # Filter out potentially dangerous division by zero or very low SBP
        shock_index_df = shock_index_df[shock_index_df['SysABP'] > 10.0]
        
        if len(shock_index_df) >= 1:
            shock_index_df['ShockIndex'] = shock_index_df['HR'] / shock_index_df['SysABP']
            features['ShockIndex_max'] = shock_index_df['ShockIndex'].max()
            
            # Calculate trend if we have enough points
            if len(shock_index_df) >= 3 and 'Time' in ts_data.columns:
                try:
                    # Merge time information
                    time_data = ts_data.loc[shock_index_df.index, 'Time']
                    if not time_data.empty:
                        x = time_data.values
                        y = shock_index_df['ShockIndex'].values
                        features['ShockIndex_trend'] = np.polyfit(x, y, 1)[0]
                except:
                    # Keep default value (0.0) if calculation fails
                    pass
    
    # 4. Additional critical composite features - 3 features
    
    # 4.1 PaO2/FiO2 ratio (for respiratory function) - 1 feature
    features['PF_ratio_min'] = 500.0  # Default to normal value (healthy is >300)
    
    if 'PaO2' in ts_data.columns and 'FiO2' in ts_data.columns:
        pf_ratio_df = ts_data[['PaO2', 'FiO2']].dropna()
        # Avoid division by zero or very low FiO2
        pf_ratio_df = pf_ratio_df[pf_ratio_df['FiO2'] > 0.2]  # FiO2 should be at least 0.21 (room air)
        
        if not pf_ratio_df.empty:
            pf_ratio_df['PF_ratio'] = pf_ratio_df['PaO2'] / pf_ratio_df['FiO2']
            features['PF_ratio_min'] = pf_ratio_df['PF_ratio'].min()
    
    # 4.2 Lactate clearance (for tissue perfusion) - 1 feature
    features['Lactate_clearance'] = 0.0  # Default to no change
    
    if 'Lactate' in ts_data.columns and 'Time' in ts_data.columns:
        lactate_df = ts_data[['Time', 'Lactate']].dropna()
        if len(lactate_df) >= 2:
            # Sort by time
            lactate_df = lactate_df.sort_values('Time')
            # Calculate lactate clearance (negative trend is good)
            try:
                x = lactate_df['Time'].values
                y = lactate_df['Lactate'].values
                features['Lactate_clearance'] = np.polyfit(x, y, 1)[0]
            except:
                # Keep default value (0.0) if calculation fails
                pass
    
    # 4.3 Glasgow Coma Scale deterioration - 1 feature
    features['GCS_deterioration'] = 0  # Default to no deterioration
    
    if 'GCS' in ts_data.columns:
        gcs_values = ts_data['GCS'].dropna()
        if len(gcs_values) >= 2:
            # Neurological deterioration = decrease in GCS of 2 or more points
            features['GCS_deterioration'] = 1 if (gcs_values.max() - gcs_values.min() >= 2) else 0
    
    # Make sure any existing NaN values from the original features are also replaced
    for key in features:
        if pd.isna(features[key]):
            # For categorical/binary features
            if key == 'Gender':
                features[key] = 'Unknown'
            elif key.endswith('_deterioration') or key == 'MechVent':
                features[key] = 0
            # For continuous features - use appropriate default values
            elif key in ['Age', 'Height', 'Weight']:
                features[key] = -1  # Indicating missing demographic
            else:
                features[key] = 0.0  # Default for other continuous variables
    
    return features


In [32]:
'''
def extract_advanced_clinical_features(patient_data):
    """
    Extract advanced clinically relevant features from patient time-series data
    using tsfresh to automatically extract and select important time-series features.
    
    Args:
        patient_data: DataFrame containing patient time-series data
        
    Returns:
        Dictionary of extracted features with no NaN values
    """
    import numpy as np
    import pandas as pd
    from tsfresh import extract_features
    from tsfresh.utilities.dataframe_functions import impute
    
    # First get all the original features
    features = extract_clinical_features(patient_data)
    
    # Make sure we have a time column - if not, create one based on index
    if 'Hours' not in patient_data.columns:
        print("no")
    else:
        ts_data = patient_data.copy()
    
    # Rename columns to match tsfresh requirements
    ts_data = ts_data.reset_index(drop=True)
    
    # Create a patient ID column if not present
    if 'PatientID' in ts_data.columns and 'id' not in ts_data.columns:
        ts_data = ts_data.rename(columns={'PatientID': 'id'})
        print("success")
    
    # Ensure the time column is properly named
    if 'Hours' in ts_data.columns and 'time' not in ts_data.columns:
        ts_data = ts_data.rename(columns={'Hours': 'time'})
        print("su")
    
    # Define the critical vital signs for time analysis
    critical_vitals = ['HR', 'SysABP', 'RespRate', 'GCS', 'SaO2']
    
    # Select only vital signs columns and required tsfresh columns
    cols_to_use = ['id', 'time'] + [col for col in critical_vitals if col in ts_data.columns]
    
    # Add other relevant clinical columns if available
    additional_cols = ['PaO2', 'FiO2', 'Lactate', 'MechVent']
    cols_to_use += [col for col in additional_cols if col in ts_data.columns]
    
    # Filter the dataframe to include only required columns that exist
    ts_data = ts_data[cols_to_use].dropna(how='all', subset=[col for col in cols_to_use if col not in ['id', 'time']])
    
    # If we have sufficient data, extract features using tsfresh
    if not ts_data.empty and len(ts_data) >= 3:
        try:
            # Extract features using tsfresh
            extracted_features = extract_features(ts_data, 
                                                 column_id='id', 
                                                 column_sort='time')
            
            # Impute missing values
            impute(extracted_features)
            
            # Convert the extracted features DataFrame to dictionary
            # and update our features dictionary
            for col in extracted_features.columns:
                # Use a simplified name for readability
                simplified_name = col.replace('__', '_')
                # Limit name length
                if len(simplified_name) > 50:
                    simplified_name = simplified_name[:50]
                
                features[simplified_name] = extracted_features[col].iloc[0]
        except Exception as e:
            print(f"Error in tsfresh feature extraction: {e}")
            # Continue with the original features if tsfresh fails
  
    
    return features
        '''

'\ndef extract_advanced_clinical_features(patient_data):\n    """\n    Extract advanced clinically relevant features from patient time-series data\n    using tsfresh to automatically extract and select important time-series features.\n    \n    Args:\n        patient_data: DataFrame containing patient time-series data\n        \n    Returns:\n        Dictionary of extracted features with no NaN values\n    """\n    import numpy as np\n    import pandas as pd\n    from tsfresh import extract_features\n    from tsfresh.utilities.dataframe_functions import impute\n    \n    # First get all the original features\n    features = extract_clinical_features(patient_data)\n    \n    # Make sure we have a time column - if not, create one based on index\n    if \'Hours\' not in patient_data.columns:\n        print("no")\n    else:\n        ts_data = patient_data.copy()\n    \n    # Rename columns to match tsfresh requirements\n    ts_data = ts_data.reset_index(drop=True)\n    \n    # Create a pat

In [33]:


def process_dataset(data_path, outcome_path):
    """
    Process a dataset file to extract features and merge with outcomes
    
    Args:
        data_path: Path to the dataset file
        outcome_path: Path to the matching outcome file
        
    Returns:
        X: Feature matrix
        y: Target vector
    """
    print(f"Processing dataset: {data_path}")
    print(f"Using outcome file: {outcome_path}")
    
    # Load data
    df = pd.read_csv(data_path)
    
    # Load outcome data
    outcome_df = pd.read_csv(outcome_path)
    
    # Rename the ID column in outcome data to match patient data
    outcome_df = outcome_df.rename(columns={'RecordID': 'PatientID'})
    
    # Keep only the PatientID and In-hospital_death columns for our prediction task
    outcome_df = outcome_df[['PatientID', 'In-hospital_death']]
    
    print(f"Number of patients in dataset: {df['PatientID'].nunique()}")
    print(f"Number of patients in outcome file: {len(outcome_df)}")
    
    # Extract features for each patient
    print("Extracting features...")
    features_list = []
    for patient_id in df['PatientID'].unique():
        patient_data = df[df['PatientID'] == patient_id]
        features = extract_advanced_clinical_features(patient_data)
        features['PatientID'] = patient_id
        features_list.append(features)
    
    # Create features dataframe
    features_df = pd.DataFrame(features_list)
    
    # Merge features with outcome data
    merged_data = pd.merge(features_df, outcome_df, on='PatientID', how='inner')
    
    print(f"Number of patients with matching outcome data: {len(merged_data)}")
    
    # Separate features and target
    X = merged_data.drop(['PatientID', 'In-hospital_death'], axis=1)
    y = merged_data['In-hospital_death']
    
    # Check for any remaining NaN values (shouldn't be any since data is imputed)
    nan_counts = X.isna().sum().sum()
    if nan_counts > 0:
        print(f"Warning: {nan_counts} NaN values found in features")
    
    return X, y


In [34]:
def main():
    # Define file paths
    train_data_path = '/home/taekim/enhanced_preprocessed_data/enhanced_sampled_set-a.csv' #used scaled version only for training set
    val_data_path = '/home/taekim/enhanced_preprocessed_data/enhanced_sampled_set-b.csv'
    test_data_path = '/home/taekim/enhanced_preprocessed_data/enhanced_sampled_set-c.csv'
    train_outcome_path = '/home/taekim/ml4h_data/p1/Outcomes-a.txt'
    val_outcome_path = '/home/taekim/ml4h_data/p1/Outcomes-b.txt'
    test_outcome_path = '/home/taekim/ml4h_data/p1/Outcomes-c.txt'
    
    # Process each dataset with its corresponding outcome file
    print("\nProcessing training data...")
    X_train, y_train = process_dataset(train_data_path, train_outcome_path)
    print("\nProcessing validation data...")
    X_val, y_val = process_dataset(val_data_path, val_outcome_path)
    print("\nProcessing test data...")
    X_test, y_test = process_dataset(test_data_path, test_outcome_path)
    
    # Check class distributions
    print("\nClass distributions:")
    print(f"Training: {dict(y_train.value_counts())}")
    print(f"Validation: {dict(y_val.value_counts())}")
    print(f"Test: {dict(y_test.value_counts())}")
    
    # Train classifiers
    print("\nTraining classifiers...")
    classifiers = {
        'Logistic Regression': LogisticRegression(max_iter=1000, class_weight='balanced', random_state=42),
        'Random Forest': RandomForestClassifier(n_estimators=100, class_weight='balanced', random_state=42),
        'XGBoost': XGBClassifier(
            n_estimators=100,
            learning_rate=0.1,
            max_depth=5,
            subsample=0.8,
            colsample_bytree=0.8,
            scale_pos_weight=len(y_train[y_train==0])/len(y_train[y_train==1]),  # For imbalanced classes
            random_state=42
        )
    }
    
    # Train and evaluate on validation set
    validation_results = {}
    for name, classifier in classifiers.items():
        print(f"\nTraining {name}...")
        
        # Train model on training data
        if name == 'XGBoost':
            # XGBoost fit with eval_set
            # Note: In newer XGBoost versions, early_stopping_rounds is passed directly
            # In older versions, it's set as an attribute before fitting
            classifier.set_params(eval_metric=['auc', 'logloss'])
            classifier.fit(
                X_train, y_train,
                eval_set=[(X_val, y_val)],
                verbose=True
            )
            # Try to set early stopping via the classifier object if needed
            if hasattr(classifier, 'best_iteration'):
                print(f"Best iteration found: {classifier.best_iteration}")
        else:
            # Standard scikit-learn fit for other models
            classifier.fit(X_train, y_train)
        
        # Evaluate on validation data
        y_val_proba = classifier.predict_proba(X_val)[:, 1]
        val_auroc = roc_auc_score(y_val, y_val_proba)
        val_auprc = average_precision_score(y_val, y_val_proba)
        
        validation_results[name] = {
            'AUROC': val_auroc,
            'AUPRC': val_auprc
        }
        
        # Show feature importances for tree-based models
        if name in ['Random Forest', 'XGBoost']:
            if name == 'Random Forest':
                importances = classifier.feature_importances_
            else:  # XGBoost
                importances = classifier.feature_importances_
                
            feature_importance = pd.DataFrame({
                'Feature': X_train.columns,
                'Importance': importances
            }).sort_values('Importance', ascending=False)
            
            print(f"\nMost important features for {name}:")
            print(feature_importance)
    
    # Print validation performance results
    print("\nClassifier Performance on Validation Data:")
    print("-" * 60)
    for name, metrics in validation_results.items():
        print(f"{name}:")
        print(f"  AUROC: {metrics['AUROC']:.4f}")
        print(f"  AUPRC: {metrics['AUPRC']:.4f}")
    print("-" * 60)
    
    # Now evaluate on test set C
    test_results = {}
    for name, classifier in classifiers.items():
        # Evaluate on test data
        y_test_proba = classifier.predict_proba(X_test)[:, 1]
        test_auroc = roc_auc_score(y_test, y_test_proba)
        test_auprc = average_precision_score(y_test, y_test_proba)
        
        test_results[name] = {
            'AUROC': test_auroc,
            'AUPRC': test_auprc
        }

        y_test_pred = (y_test_proba >= 0.5).astype(int)
        plot_and_save_confusion_matrix(
            y_test, y_test_pred,
            title=f"{name} - Confusion Matrix (Test)",
            filename=f"conf_matrix_test_{name.replace(' ', '_').lower()}.png"
        )
    
    # Print test performance results
    print("\nClassifier Performance on Test Set C:")
    print("-" * 60)
    for name, metrics in test_results.items():
        print(f"{name}:")
        print(f"  AUROC: {metrics['AUROC']:.4f}")
        print(f"  AUPRC: {metrics['AUPRC']:.4f}")
    print("-" * 60)


    

if __name__ == "__main__":
    # Required imports
    import pandas as pd
    import numpy as np
    from sklearn.linear_model import LogisticRegression
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.metrics import roc_auc_score, average_precision_score
    from xgboost import XGBClassifier  # Make sure to import XGBoost
    
    main()


Processing training data...
Processing dataset: /home/taekim/enhanced_preprocessed_data/enhanced_sampled_set-a.csv
Using outcome file: /home/taekim/ml4h_data/p1/Outcomes-a.txt
Number of patients in dataset: 4000
Number of patients in outcome file: 4000
Extracting features...
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su
su


/cluster/courses/ml4h/project1env/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(



Training Random Forest...

Most important features for Random Forest:
                 Feature  Importance
48             GCS_trend    0.075960
37                 Urine    0.051339
0                    Age    0.035085
17                  HCO3    0.032554
16               Glucose    0.030953
24               Lactate    0.024330
13                 NIMAP    0.024303
43       GCS_variability    0.023038
40        HR_variability    0.021298
14                   BUN    0.021282
36              MechVent    0.020469
4                     HR    0.020358
3                 Weight    0.020071
53     Lactate_clearance    0.018441
42  RespRate_variability    0.018349
30                   AST    0.018312
10                  SaO2    0.018282
26                 PaCO2    0.017994
8               RespRate    0.017845
31                   ALP    0.017513
25                    pH    0.017409
29                   ALT    0.017358
32             Bilirubin    0.017177
11              NISysABP    0.017033
34  